In [1]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import os
from pathlib import Path
import pandas as pd
from astropy.time import Time
from astropy.coordinates import Distance
from astropy import units as u
import numpy as np
from astropy.cosmology import WMAP9 as cosmo
from astropy import constants as const
from matplotlib.animation import FuncAnimation
from mpl_toolkits.axes_grid1.inset_locator import zoomed_inset_axes, mark_inset

In [2]:
base_dir = Path().absolute()
data_dir = os.path.join(base_dir, "data")

In [3]:
t_peak_mjd = Time(58603.87, format="mjd")
t_neutrino = Time("2019-10-01T20:09:18.17", format='isot', scale='utc')

bran_z = 0.0512

xpath = os.path.join(data_dir, "BranStark.dat")
# with open(xpath, "r") as f:
#     print(f.read())

In [4]:
df = pd.read_table(xpath, skiprows=4, sep="\s+")
print(df)

     #day_since_peak   band   flux_Jy       nu_rest           lum  \
0           15.54514   UVW2  0.000522  1.569084e+15  9.479170e+43   
1           19.23147   UVW2  0.000472  1.569084e+15  8.565847e+43   
2           22.20333   UVW2  0.000426  1.569084e+15  7.740523e+43   
3           25.16893   UVW2  0.000415  1.569084e+15  7.529572e+43   
4           28.24098   UVW2  0.000418  1.569084e+15  7.599243e+43   
..               ...    ...       ...           ...           ...   
212         34.81478  r.ZTF  0.000159  4.961606e+14  5.833004e+42   
213         38.61815  r.ZTF  0.000151  4.961606e+14  5.556640e+42   
214         44.28862  r.ZTF  0.000145  4.961606e+14  5.302641e+42   
215         47.13955  r.ZTF  0.000138  4.961606e+14  5.057923e+42   
216         56.65696  r.ZTF  0.000125  4.961606e+14  4.573959e+42   

          err_lum        lum_kc        lum_bb  
0    2.619191e+42  1.218535e+43  2.686519e+44  
1    4.733662e+42  1.113457e+43  2.369954e+44  
2    3.564643e+42  1.015059

In [5]:
meerkat_path = os.path.join(data_dir, "at2019dsg_MeerKAT.txt")
meerkat_data = pd.read_table(meerkat_path, sep="\s+")
print(meerkat_data)

       #mjd           date  flux_mJy  flux_err_mJy
0  58653.08  2019-Jun-19.1     0.104         0.018
1  58694.00  2019-Jul-30.0     0.111         0.017
2  58761.79  2019-Oct-05.8     0.152         0.019


In [6]:
radio_data = [x for x in os.listdir(data_dir) if "at2019dsg_20" in x]
print(radio_data)
vla_data = []
for x in radio_data:
    with open(os.path.join(data_dir, x), "r") as f:
        print(x)
        rawdate = (x.split("_")[1][:8])
        obs_date = Time("{0}-{1}-{2}T00:00:00.00".format(rawdate[:4], rawdate[4:6], rawdate[6:]), format='isot', scale='utc')
        for line in f.readlines():
            line = line.replace("\n", "")
            vla_data.append(tuple([obs_date.mjd] +  [float(x) for x in line.split(" ")]))
            
vla_data = pd.DataFrame(vla_data, columns=["mjd", "frequency", "flux", "flux_err"])
print(vla_data)
#             vla_data.append()
#         print(f.read())

['at2019dsg_20190808.txt', 'at2019dsg_20190522.txt', 'at2019dsg_20191005.txt', 'at2019dsg_20171008.txt', 'at2019dsg_20190619.txt']
at2019dsg_20190808.txt
at2019dsg_20190522.txt
at2019dsg_20191005.txt
at2019dsg_20171008.txt


ValueError: could not convert string to float: '#'

In [ ]:
# xray_data_rough = np.array([
#     [17., 3. * 10 ** 43],
#     [25., 2. * 10 ** 43],
#     [30., 8. * 10 ** 42],
#     [38., 4 * 10 ** 42],
#     [45., 3 * 10 ** 42]
# ]).T

xray_path = os.path.join(data_dir, "bran_lx_Swift.dat")
xray_data = pd.read_table(xray_path, sep="\s+")
print(xray_data)

In [ ]:
gamma_path = os.path.join(data_dir, "TDE_uls_FermiLAT")
gamma_data = pd.read_table(gamma_path, sep=",")
print(gamma_data)

In [ ]:
# Formula from https://arxiv.org/abs/1510.01226

print(Distance(z=bran_z), cosmo.luminosity_distance(z=bran_z))

peak_f = []
peak_flux = []
dates = []
for date in sorted(list(set(vla_data["mjd"]))):
    data = vla_data[vla_data["mjd"] == date]
    max_index = list(data["flux"]).index(max(data["flux"]))
    peak_f.append(list(data["frequency"])[max_index])
    peak_flux.append(max(data["flux"]))
    dates.append(date)
dates = np.array(dates)
    
peak_f = np.array(peak_f)
peak_flux = np.array(peak_flux)

# f_a = 1.0 for spherical or 0.1 for conical

# Volume is shell of radius 0.1
f_v = 4./3. * (1.**3 - 0.9**3.)

# volume_of_cone = 4/3 pi r^3 times ratio of cone area to sphere surface area
# Volume of cone is thus f_v * f_a

# Works for spherical case now :)

def equipartition_radius(peak_f_ghz, peak_flux_mjy, f_a=1.0, z=bran_z):
    return (3.2 * 10**15) * u.cm * (
        (peak_flux_mjy ** (9./19.)) * 
        ((cosmo.luminosity_distance(z=z).to(u.cm)/(u.cm * 10**26))**(18./19.)) *
        ((peak_f_ghz/10.) ** -1.) * 
        ((1 + z) ** (-10./19.)) * 
        (f_a ** (-8./19.)) * 
        ((f_v * f_a) ** (-1./19.))
    ) * 4.**(1./19.)
    
def equipartition_energy(peak_f_ghz, peak_flux_mjy, f_a=1.0, z=bran_z):
    return (1.9 * 10**46) * u.erg * (
        (peak_flux_mjy ** (23./19.)) * 
        ((cosmo.luminosity_distance(z=z).to(u.cm)/(u.cm * 10**26))**(46./19.)) *
        ((peak_f_ghz/10.) ** -1.) * 
        ((1 + z) ** (-42./19.)) * 
        (f_a ** (-12./19.)) * 
        ((f_v * f_a) ** (8./19.))
    )* 4.**(11./19.)

# Cross check with table 2 of the paper, using row 2 of the spherical section

print("Checking formula using values from https://arxiv.org/abs/1510.01226")

asassn_14li_z = 0.0206
print("Distance to ASASSN-14li: {0:.2f} \n".format(Distance(z=asassn_14li_z).to(u.Mpc)))

published_r_s = 1.47 * 10**16 * u.cm
calc_r_s = equipartition_radius(8.20, 1.76, z=asassn_14li_z)
published_e_s = 7.8 * 10**47 * u.erg
calc_e_s = equipartition_energy(8.20, 1.76, z=asassn_14li_z)

published_r_c = 4.37 * 10**16 * u.cm
calc_r_c = equipartition_radius(8.20, 1.76, z=asassn_14li_z, f_a=0.1)
published_e_c = 3.19 * 10**47 * u.erg
calc_e_c = equipartition_energy(8.20, 1.76, z=asassn_14li_z, f_a=0.1)

# published_e = 9.5 * 10**47 * u.erg
# calc_e = equipartition_energy(4.37, 1.23, z=asassn_14li_z)

print("Spherical Case")

print("Checking with ASSASSN-14li, we should have Equipartition Radius of {0:.3g}.".format(published_r_s))
print("We find {0:.3g}, a ratio of {1:.2f}".format(calc_r_s, calc_r_s/published_r_s))
print("Checking with ASSASSN-14li, we should have Equipartition Energy of {0:.3g}.".format(published_e_s))
print("We find {0:.3g}, a ratio of {1:.2f} \n".format(calc_e_s, calc_e_s/published_e_s))

print("Conical Case")

print("Checking with ASSASSN-14li, we should have Equipartition Radius of {0:.3g}.".format(published_r_c))
print("We find {0:.3g}, a ratio of {1:.2f}".format(calc_r_c, calc_r_c/published_r_c))
print("Checking with ASSASSN-14li, we should have Equipartition Energy of {0:.3g}.".format(published_e_c))
print("We find {0:.3g}, a ratio of {1:.2f}".format(calc_e_c, calc_e_c/published_e_c))

In [ ]:
def get_delta_times(times=dates):
    return (times[1:] - times[:-1]) * u.day

def calc_av_expansion(f, flux, times=dates, z=bran_z, f_a=1):
    delta_t = get_delta_times(times)
    rads = equipartition_radius(f, flux, f_a=f_a, z=z)
    delta_rad = rads[1:] - rads[:-1]
    vel = (delta_rad / delta_t).to("m s-1")
    return vel

delta_times = get_delta_times()

# beta = vel/(3 * 10 ** 8 *u.m/u.s)
# print(beta)
# print(vel.to("km s-1"))
offset_times = dates[:-1] + 0.5 * get_delta_times(dates) / u.day
# print(offset_times)

In [ ]:
bands = list(set(df["band"]))
print(bands)

In [ ]:
def convert_radio(flux_mjy, frequency_ghz):
    flux_jy = 10**-3 * flux_mjy
    frequency_hz = 10**9 * frequency_ghz
    return 10**-23 * flux_jy * frequency_hz

def convert_to_mjy(energy_flux, frequency_ghz):
    frequency_hz = 10**9 * frequency_ghz
    return 10 ** 3 * 10 ** 23 * energy_flux / frequency_hz

print(convert_to_mjy(convert_radio(1., 1.), 1.))


In [ ]:
plt.figure(figsize=(12, 20))

ax1 = plt.subplot(611)

dl = Distance(z=bran_z).to("cm").value
area = 4 * np.pi * (dl ** 2)

conversion = 1./area

colors = {
    "r.IOO": "r",
    "r.ZTF": "r",
    "r.SEDM": "r",
    "g.ZTF": "g",
    "g.IOO": "g"
}

# Plot luminosity

for band in sorted(bands):
    data = df[df["band"] == band]
    
    if band in colors:
        c = colors[band]
    else:
        c = None
    
    ax1.errorbar(data["#day_since_peak"], conversion * data["lum"], yerr=conversion *data["err_lum"], color=c,  fmt='o', label=band)

# ax1.plot(meerkat_data["#mjd"] - t_peak_mjd.mjd, convert_radio(meerkat_data["flux_mJy"], 1.4), marker="o", color="violet", label="1.4 GHz")

# for frequency in [10.2]:
#     data = vla_data[abs(vla_data["frequency"] - frequency) < 0.5]
#     data = data.sort_values("mjd")
#     ax1.errorbar(data["mjd"]- t_peak_mjd.mjd, convert_radio(data["flux"], frequency),  yerr=convert_radio(data["flux_err"], frequency), marker="o", label="{0} Ghz".format(frequency))

# ax1.set_ylabel("Luminosity", fontsize=12)
ax1.set_ylabel(r"Energy Flux [erg cm$^{-2}$ s$^{-1}$]", fontsize=12)
ax1.legend(fontsize=12)
ax1.set_yscale("log")
ax1.axvline(t_neutrino.mjd - t_peak_mjd.mjd, color="k", linestyle=":", label="IC191001A")
ax1.tick_params(axis='both', which='major', labelsize=12)

# Plot Black Body Luminosity

ax2 = plt.subplot(612, sharex=ax1)
ax2.scatter(df["#day_since_peak"][df["lum_bb"] > 0], df["lum_bb"][df["lum_bb"] > 0], color="green", label="Black Body Luminosity")

ax2.set_yscale("log")
ax2.axvline(t_neutrino.mjd - t_peak_mjd.mjd, color="k", linestyle=":")
ax2.set_ylabel("BB Luminosity", fontsize=12)
ax2.legend(fontsize=12)
ax2.tick_params(axis='both', which='major', labelsize=12)


# Radio Data

ax3 = plt.subplot(613, sharex=ax1)
ax3.plot(meerkat_data["#mjd"] - t_peak_mjd.mjd, meerkat_data["flux_mJy"], marker="o", color="violet", label="1.4 GHz")
ax3.axvline(t_neutrino.mjd - t_peak_mjd.mjd, color="k", linestyle=":")

for frequency in [5.6, 10.2]:
    data = vla_data[abs(vla_data["frequency"] - frequency) < 0.5]
    data = data.sort_values("mjd")
    ax3.errorbar(data["mjd"]- t_peak_mjd.mjd, data["flux"],  yerr=data["flux_err"], marker="o", label="{0} Ghz".format(frequency))

ax3.legend(fontsize=12)
ax3.set_ylabel("Radio Flux (mJy)", fontsize=12)
ax3.set_yscale("log")
ax3.tick_params(axis='both', which='major', labelsize=12)

# Equipartition Energy

ax4 = plt.subplot(614, sharex=ax1)

# ax4.plot(dates - t_peak_mjd.mjd, equipartition_energy(peak_f, peak_flux),  marker="o", color="gray", label=r"Spherical ($f_{A}=1.0$)")
# ax4.plot(dates - t_peak_mjd.mjd, equipartition_energy(peak_f, peak_flux, f_a=0.1),  marker="*", linestyle=":", color="gray", label=r"Conical ($f_{A}=0.1$)")
ax4.plot(dates - t_peak_mjd.mjd, equipartition_energy(peak_f, peak_flux),  marker="o", color="red", label=r"Spherical ($f_{A}=1.0$)")
# ax4.plot(dates - t_peak_mjd.mjd, equipartition_energy(peak_f, peak_flux, f_a=0.1),  marker="*", color="blue", label=r"Conical ($f_{A}=0.1$)")
ax4.axvline(t_neutrino.mjd - t_peak_mjd.mjd, color="k", linestyle=":")
ax4.set_yscale("log")
ax4.set_ylabel("Equipartition Energy (erg)", fontsize=12)
ax4.legend(fontsize=12)
ax4.tick_params(axis='both', which='major', labelsize=12)

# Equipartition Radius

ax5 = plt.subplot(615, sharex=ax1)
# ax5.plot(dates - t_peak_mjd.mjd, equipartition_radius(peak_f, peak_flux), marker="o", color="brown", label=r"Spherical ($f_{A}=1.0$)")
# ax5.plot(dates - t_peak_mjd.mjd, equipartition_radius(peak_f, peak_flux, f_a=0.1),  marker="*", linestyle=":", color="brown", label=r"Conical ($f_{A}=0.1$)")
ax5.plot(dates - t_peak_mjd.mjd, equipartition_radius(peak_f, peak_flux), marker="o", color="red", label=r"Spherical ($f_{A}=1.0$)")
# ax5.plot(dates - t_peak_mjd.mjd, equipartition_radius(peak_f, peak_flux, f_a=0.1),  marker="*", color="blue", label=r"Conical ($f_{A}=0.1$)")

ax5.axvline(t_neutrino.mjd - t_peak_mjd.mjd, color="k", linestyle=":")
ax5.set_yscale("log")
ax5.set_ylabel("Equipartition Radius (cm)", fontsize=12)
ax5.legend(fontsize=12)
ax5.tick_params(axis='both', which='major', labelsize=12)


# Average Expansion Velocity


ax6 = plt.subplot(616, sharex=ax1)
# ax6.errorbar(offset_times - t_peak_mjd.mjd, [x.value for x in calc_av_expansion(peak_f, peak_flux, f_a=1.0)/(3 * 10 ** 8)], xerr=0.5 * delta_times / u.day, fmt="o", color="blue", label=r"Spherical ($f_{A}=1.0$)")
# eb1 = ax6.errorbar(offset_times - t_peak_mjd.mjd, [x.value for x in calc_av_expansion(peak_f, peak_flux, f_a=0.1)/(3 * 10 ** 8)], xerr=0.5 * delta_times / u.day, fmt="o", color="blue", marker="*", label=r"Conical ($f_{A}=0.1$)")
# eb1[-1][0].set_linestyle(':')
ax6.errorbar(offset_times - t_peak_mjd.mjd, [x.value for x in calc_av_expansion(peak_f, peak_flux, f_a=1.0)/(3 * 10 ** 8)], xerr=0.5 * delta_times / u.day, fmt="o", color="red", label=r"Spherical ($f_{A}=1.0$)")
# eb1 = ax6.errorbar(offset_times - t_peak_mjd.mjd, [x.value for x in calc_av_expansion(peak_f, peak_flux, f_a=0.1)/(3 * 10 ** 8)], xerr=0.5 * delta_times / u.day, fmt="o", color="blue", marker="*", label=r"Conical ($f_{A}=0.1$)")

ax6.axvline(t_neutrino.mjd - t_peak_mjd.mjd, color="k", linestyle=":")
ax6.set_ylabel(r"Avg dR/dt (c=1)", fontsize=12)
ax6.legend(fontsize=12)

plt.xlabel("Time since peak (days)", fontsize=12)
ax6.tick_params(axis='both', which='major', labelsize=12)

plt.tight_layout()
plt.subplots_adjust(hspace=.0)

In [ ]:
plt.figure(figsize=(12, 7))

# Radio Data

ax3 = plt.subplot(111)
ax3.plot(meerkat_data["#mjd"] - t_peak_mjd.mjd, meerkat_data["flux_mJy"], marker="o", color="violet", label="1.4 GHz")
ax3.axvline(t_neutrino.mjd - t_peak_mjd.mjd, color="k", linestyle=":")

for frequency in [5.6, 10.2]:
    data = vla_data[abs(vla_data["frequency"] - frequency) < 0.5]
    data = data.sort_values("mjd")
    ax3.errorbar(data["mjd"]- t_peak_mjd.mjd, data["flux"],  yerr=data["flux_err"], marker="o", label="{0} Ghz".format(frequency))

ax3.legend(fontsize=12)
ax3.set_ylabel("Radio Flux (mJy)", fontsize=12)
ax3.set_yscale("log")
ax3.tick_params(axis='both', which='major', labelsize=12)
plt.xlabel("Time since peak (days)", fontsize=12)


In [ ]:
asassn_14li_data = np.array([
    [143., 8.20, 1.76],
    [207., 4.37, 1.23],
    [246., 4.00, 1.14],
    [304., 2.55, 0.94]
])

# First epoch on 24 dec
# TDE discovered 22 Nov

asassn_14li_data.T[0] -= 143.
asassn_14li_data.T[0] += 52.

In [ ]:
plt.figure(figsize=(12, 10))

ax1 = plt.subplot(311)

dl = Distance(z=bran_z).to("cm").value
area = 4 * np.pi * (dl ** 2)

conversion = 1./area

# Equipartition Energy

ax4 = plt.subplot(311)

# ax4.plot(dates - t_peak_mjd.mjd, equipartition_energy(peak_f, peak_flux),  marker="o", color="gray", label=r"Spherical ($f_{A}=1.0$)")
# ax4.plot(dates - t_peak_mjd.mjd, equipartition_energy(peak_f, peak_flux, f_a=0.1),  marker="*", linestyle=":", color="gray", label=r"Conical ($f_{A}=0.1$)")
ax4.plot(
    dates - t_peak_mjd.mjd,
    equipartition_energy(peak_f, peak_flux),
    marker="o",
    color="red",
    label=r"ZTF19aapreis (Spherical)")
# ax4.plot(dates - t_peak_mjd.mjd, equipartition_energy(peak_f, peak_flux, f_a=0.1),  marker="*", color="blue", label=r"Conical ($f_{A}=0.1$)")

ax4.plot(
    asassn_14li_data.T[0],
    equipartition_energy(asassn_14li_data.T[1], asassn_14li_data.T[2], z=asassn_14li_z),
    marker="o",
    color="blue",
    label=r"ASASSN-14li (Spherical)"
)

ax4.axvline(t_neutrino.mjd - t_peak_mjd.mjd, color="r", linestyle=":", label="IC191001A")
ax4.set_yscale("log")
ax4.set_ylabel("Energy [erg]", fontsize=16)
ax4.legend(fontsize=16)
ax4.tick_params(axis='both', which='major', labelsize=16)

# Equipartition Radius

ax5 = plt.subplot(312, sharex=ax4)
# ax5.plot(dates - t_peak_mjd.mjd, equipartition_radius(peak_f, peak_flux), marker="o", color="brown", label=r"Spherical ($f_{A}=1.0$)")
# ax5.plot(dates - t_peak_mjd.mjd, equipartition_radius(peak_f, peak_flux, f_a=0.1),  marker="*", linestyle=":", color="brown", label=r"Conical ($f_{A}=0.1$)")
ax5.plot(
    dates - t_peak_mjd.mjd,
    [x.value for x in equipartition_radius(peak_f, peak_flux)],
    marker="o",
    color="red",
    label=r"Spherical ($f_{A}=1.0$)"
)
ax5.plot(
    asassn_14li_data.T[0],
    [x.value for x in equipartition_radius(asassn_14li_data.T[1], asassn_14li_data.T[2], z=asassn_14li_z)],
    marker="o",
    color="blue",
    label=r"Spherical ($f_{A}=1.0$)"
)
print(equipartition_radius(asassn_14li_data.T[1], asassn_14li_data.T[2], z=asassn_14li_z))
# ax5.plot(dates - t_peak_mjd.mjd, equipartition_radius(peak_f, peak_flux, f_a=0.1),  marker="*", color="blue", label=r"Conical ($f_{A}=0.1$)")

ax5.axvline(t_neutrino.mjd - t_peak_mjd.mjd, color="r", linestyle=":", label="IC191001A")
ax5.set_yscale("log")
ax5.set_ylabel("Radius [cm]", fontsize=16)
# ax5.legend(fontsize=16)
ax5.tick_params(axis='both', which='major', labelsize=16)
# ax5.ticklabel_format(axis='both', style="sci")


# Average Expansion Velocity


ax6 = plt.subplot(313, sharex=ax4)
# ax6.errorbar(offset_times - t_peak_mjd.mjd, [x.value for x in calc_av_expansion(peak_f, peak_flux, f_a=1.0)/(3 * 10 ** 8)], xerr=0.5 * delta_times / u.day, fmt="o", color="blue", label=r"Spherical ($f_{A}=1.0$)")
# eb1 = ax6.errorbar(offset_times - t_peak_mjd.mjd, [x.value for x in calc_av_expansion(peak_f, peak_flux, f_a=0.1)/(3 * 10 ** 8)], xerr=0.5 * delta_times / u.day, fmt="o", color="blue", marker="*", label=r"Conical ($f_{A}=0.1$)")
# eb1[-1][0].set_linestyle(':')
ax6.errorbar(
    offset_times - t_peak_mjd.mjd,
    [x.value for x in calc_av_expansion(peak_f, peak_flux, f_a=1.0)/(3 * 10 ** 8)],
    xerr=0.5 * get_delta_times(dates) / u.day,
    fmt="o",
    color="red",
    label=r"Spherical ($f_{A}=1.0$)"
)

y = [
    g.value for g in calc_av_expansion(
        asassn_14li_data.T[1],
        asassn_14li_data.T[2],
        times=asassn_14li_data.T[0],
        z=asassn_14li_z,
        f_a=1.0,
    )/(3*10**8)
]

print(y)

t = np.array([x.value for x in get_delta_times(asassn_14li_data.T[0])])
offset_t = asassn_14li_data.T[0][:-1] + 0.5 * t
print(t)



ax6.errorbar(
    offset_t,
    y,
    xerr=0.5 * get_delta_times(asassn_14li_data.T[0]) / u.day,
    fmt="o",
    color="blue",
    label=r"Spherical ($f_{A}=1.0$)"
)
# eb1 = ax6.errorbar(offset_times - t_peak_mjd.mjd, [x.value for x in calc_av_expansion(peak_f, peak_flux, f_a=0.1)/(3 * 10 ** 8)], xerr=0.5 * delta_times / u.day, fmt="o", color="blue", marker="*", label=r"Conical ($f_{A}=0.1$)")

ax6.axvline(t_neutrino.mjd - t_peak_mjd.mjd, color="r", linestyle=":", label="IC191001A")
ax6.set_ylabel(r"Expansion Rate (c=1)", fontsize=16)
# ax6.legend(fontsize=16)
ax6.set_ylim(bottom=0.0)

plt.xlabel("Time since peak (days)", fontsize=16)
ax6.tick_params(axis='both', which='major', labelsize=16)

# plt.tight_layout()
plt.subplots_adjust(hspace=.0)

In [ ]:
dl = Distance(z=bran_z).to("cm").value
area = 4 * np.pi * (dl ** 2)

conversion = 1./area

In [ ]:
# print(xray_data["flux"], 10**xray_data["log_lum"], (10**xray_data["log_lum"])*conversion)

# plt.plt()

In [ ]:
radio_times = vla_data.sort_values("mjd")["mjd"] - t_peak_mjd.mjd

plt.figure(figsize=(12,10))
ax = plt.subplot(111)

tnt = 157.12999999999738

mask = radio_times == tnt
data = vla_data[mask]

convert_y = convert_radio(data["flux"], data["frequency"])
convert_y_err = convert_radio(data["flux_err"], data["frequency"])

ax.errorbar(
    data["frequency"] * 10. ** 9, 
    convert_y, 
    marker="o", 
    yerr=convert_radio(data["flux_err"], data["frequency"]),
    fmt = "",
    linestyle=" "
)

f_mask = data["frequency"] > 5.
f_data = data[f_mask]
convert_y = list(convert_y[f_mask])
convert_y_err = list(convert_y_err[f_mask])
grad = (np.log(convert_y[-2]) - np.log(convert_y[0]))/(
    np.log(list(f_data["frequency"])[-2]) - np.log(list(f_data["frequency"])[0])
)


def radio_photon(f):
    return np.exp(np.log(convert_y[0]) + grad * (np.log(f) - np.log(list(f_data["frequency"])[0])))

x = np.array([min(f_data["frequency"]), max(f_data["frequency"])*3.])
print(x, radio_photon(x), grad)

plt.plot(x*10.**9, radio_photon(x), color="k", linestyle=":")

ax.tick_params(axis='both', which='major', labelsize=12)
plt.xscale("log")
plt.yscale("log")
plt.ylabel(r"Flux erg cm{-2}", fontsize=12)
plt.xlabel("f (Hz)", fontsize=12)

In [ ]:
print(convert_y, x)
y_rad = np.array(convert_y)
yerr_rad = np.array(convert_y_err) 
# yerr_rad = np.log(base_yerr + np.exp(y)) - y
# print(yerr, type(yerr))
x_rad = np.array(list(f_data["frequency"]))


plt.errorbar(x_rad*10**9, y_rad, yerr=yerr_rad, marker="o", linestyle=" ")

# def chi2(p):
#     m = p[0]
#     c = p[1]
#     exp = m*np.log(x_rad) + c
# #     delta = np.array(np.sum(np.exp(y) - exp)**2 / np.log(yerr)**2))
#     delta = np.array(np.sum((np.exp(y) - np.exp(exp))**2)/(0.03)**2)
#     return delta

def g(x, a, b):
    
    exp = a * np.exp(b * np.log(x))

    return exp

from scipy import optimize

popt, pcov = optimize.curve_fit(g, x_rad, y_rad, sigma=yerr_rad, p0=[y_rad[0], 0.1], absolute_sigma=True)
perr = np.sqrt(np.diag(pcov))
print(popt)
print(perr)

def fit_g(x, sd=0.):
    
    sig = perr[1]
    
    m = popt[1] + sd*sig
    
    def h(x, a):
    
        exp = a * np.exp(m * np.log(x))

        return exp
    
    popt_2, _ = optimize.curve_fit(h, x_rad, y_rad, sigma=yerr_rad,  p0=[y_rad[0]], absolute_sigma=True)
    
    exp = popt_2 * np.exp(m * np.log(x))

    return exp

# p0 = (grad, convert_y[0])
# res = optimize.minimize(chi2, p0)
# print(res)
# sigma_m = np.sqrt(res.hess_inv[0][0])
# print(f"sigma {sigma_m}")

# def fit_f(x, sd=0.):
#     return np.exp((res.x[0] + sd*sigma_m)*np.log(x) + res.x[1])

plt.plot(x_rad*10.**9, fit_g(x_rad), color="k", linestyle=":")
plt.fill_between(x_rad*10.**9, fit_g(x_rad, 1), fit_g(x_rad, -1), alpha=0.1, color="k")
plt.fill_between(x_rad*10.**9, fit_g(x_rad, 2), fit_g(x_rad, -2), alpha=0.1, color="k")

# plt.plot(x_rad*10.**9, fit_g(x_rad, -1))
# plt.plot(x_rad*10.**9, fit_g(x_rad, 2))
# plt.plot(x_rad*10.**9, fit_g(x_rad, -2))
plt.yscale("log")
plt.xscale("log")

print("GRAD", grad)
# grad = res.x[0]

# def radio_photon(f):
#     return np.exp(np.log(convert_y[0]) + grad * (np.log(f) - np.log(list(f_data["frequency"])[0])))

In [ ]:

colors = {
    "r.IOO": "r",
    "r.ZTF": "r",
    "r.SEDM": "r",
    "g.ZTF": "g",
    "g.IOO": "g",
    "UVW2": "violet",
    "UVM2": "purple",
    "UVW1": "darkblue",
    "U": "lightblue",
    
}

times = []
delta_lum = []

bands = {
    "U": 3465 * u.angstrom,
    "UVW1": 2600 * u.angstrom,
    "UVM2": 2246 * u.angstrom,
    "UVW2": 1928 * u.angstrom,
    "g.ZTF": 464 * u.nm,
    "r.ZTF": 658 * u.nm,
    "r.SEDm": 658 * u.nm,
}

fig = plt.figure(figsize=(14, 6))

start_bin = 5

photon_index = []
index_times = []

ls = []

cmap = "plasma"

# def col(times):
#     frac = np.array((times)/time_scale)
#     return frac
#     return matplotlib.cm.get_cmap(cmap)(frac)


ax = fig.add_axes([0.1, 0.1, 0.6, 0.7]) 
ax.set_yscale("log")
ax.set_xscale("log")
lower_x = 1.0 * 10**8.5
upper_x = 5. * 10 ** 32
ax.set_xlim(lower_x, upper_x)
ax2 = ax.twiny()
ax2.set_xlim((u.Hz * lower_x * const.h).to("eV").value, (u.Hz * upper_x*const.h).to("eV").value)
ax2.set_xscale("log") 
lower_y = 10.**36
upper_y = 10.**46
ax.set_ylim(lower_y, upper_y)
ax3 = ax.twinx()
ax3.set_yscale("log")
ax3.set_ylim(lower_y*conversion, upper_y*conversion)
ax3.set_ylabel(r"$\nu F_{\nu}$ [erg s$^{-1}$ cm$^{-2}$]", fontsize=12)
cbaxes = fig.add_axes([0.8, 0.1, 0.03, 0.8]) 
cbar = plt.colorbar(matplotlib.cm.ScalarMappable(cmap=cmap), extend='both', cax = cbaxes)
ax.tick_params(axis='both', which='major', labelsize=12)
ax2.tick_params(axis='both', which='major', labelsize=12)
plt.suptitle("SED of AT2019dsg", fontsize=16)
ax.set_xlabel("f (Hz)", fontsize=12)
ax2.set_xlabel("E (eV)", fontsize=12)
ax.set_ylabel(r"$\nu L_{\nu}$ [erg s$^{-1}$]", fontsize=12)

x = [0.1 * 10**9, 800. * 10**9]
lim = (1.2091246077564326e-06 * u.MeV /u.cm**2 / u.s).to("erg cm-2 s-1").value/conversion
y = lim * np.ones_like(x)

ax2.errorbar(x, y, yerr=0.5*y, uplims=True, color="k")

x = [200 * 10**12, 10*10.**15]
ul_txs = 3.2 * 10 **-11./conversion
ul_1year_ehe = ul_txs *7.5
ratio_ehe_gold = 6.6 / 2.1 # ICRC REALTIME PAPER RATIO GOLD TO EHE
ul_1year_gold = ul_1year_ehe / ratio_ehe_gold

y = ul_1year_gold * np.ones_like(x)

ax2.errorbar(x, y, yerr=y*0.5, uplims=True, color="r")
ax2.plot(x, y/10.,  color="r", linestyle=":")
ax2.plot(x, y/100., color="r", linestyle=":")

frames = 80


def update_plot(j):
    
    bins = np.linspace(min(df["#day_since_peak"]), max(df["#day_since_peak"]) + 50., frames + 1)
    time_scale = bins[-1] - bins[0] 
    lower = bins[j]
    upper = bins[j + 1]
    
    t = np.mean([lower, upper])
    cbar.set_ticks([
        0.0,
        (t-bins[0])/time_scale,
        -bins[0]/time_scale,
        (t_neutrino.mjd - t_peak_mjd.mjd -bins[0])/time_scale,  
        1.0
    ])
    cbar.set_ticklabels([
        f"{bins[0]:.0f} days",
        f"T ({t:.0f} days)",
        r"$T_{peak}$ (0 days)",
        r"$T_{\nu}$" + "(+{0:.0f} days)".format(t_neutrino.mjd - t_peak_mjd.mjd),
        f"+{bins[-1]:.0f} days"
    ])
        
    mask = np.logical_and(
        df["#day_since_peak"] > lower,
        df["#day_since_peak"] < upper,
    )
                
    data = df[mask]
    
    fs = []
    lums = []

    for band in list(set(data["band"][mask])):
        if band in bands:
            wl = bands[band].to("m")
            f = (const.c / wl).to("Hz")
            
            bmask = np.logical_and(
                data["band"] == band,
                data["lum"] > 0.
            )

            bd = data[bmask]["lum"]
#             err = data[data["band"] == band]["lum"]
#             bd = np.array(list(bd[bd > 0]))
                
            x = np.ones_like(bd) * f.value
            if len(bd) > 0:
                ax.scatter(
                    x,
                    bd,
                    c=data[bmask]["#day_since_peak"],
                    cmap=cmap,
                    vmin=bins[0],
                    vmax=bins[-1]
                ) 
                fs.append(f.value)
                lums.append(np.mean(bd))
            
    radio_times = vla_data.sort_values("mjd")["mjd"] - t_peak_mjd.mjd
    
    try:
        min_x = min(fs)
        max_x = max(fs)
    except ValueError:
        min_x = 1.
        max_x = 1.
        
    mask = np.logical_and(
        radio_times > lower,
        radio_times < upper
    )
        
    if np.sum(mask) > 0:
                
        ax.scatter(
            vla_data["frequency"][mask] * 10 ** 9, 
            convert_radio(vla_data["flux"], vla_data["frequency"])[mask]/conversion, 
            marker="o", 
            c=radio_times[mask], 
            cmap=cmap, 
            vmin=bins[0],
            vmax=bins[-1]
        )

#         ax.errorbar(
#             vla_data["frequency"][mask] * 10 ** 9, 
#             convert_radio(vla_data["flux"], vla_data["frequency"])[mask]/conversion,  
#             yerr=convert_radio(vla_data["flux_err"], vla_data["frequency"])[mask]/conversion, 
#             marker="o", 
#             color=list(c)
#         )
        min_x = min(list(vla_data["frequency"][mask])) * 10 ** 9
        
    meerkat_times = meerkat_data["#mjd"] - t_peak_mjd.mjd
    
    mask = np.logical_and(
        meerkat_times > lower,
        meerkat_times < upper
    )
        
    if np.sum(mask) > 0:
        f = 1.4 * 10 ** 9
        
        ax.scatter(
            f,
            convert_radio(meerkat_data["flux_mJy"], f*10**-9)[mask]/conversion, 
            marker="o", 
            c=meerkat_times[mask], 
            cmap=cmap, 
            vmin=bins[0],
            vmax=bins[-1]
        )
        min_x = min([f, min_x])
        
    xray_times = xray_data["#MJD"] - t_peak_mjd.mjd

        
    mask = np.logical_and(
        xray_times > lower,
        xray_times < upper
    )
    
    if np.sum(mask) > 0:
        f = ((5 * u.keV) / const.h).to("Hz").value
        
        y = (10 ** xray_data["log_lum"])[mask]
        x = np.ones_like(y) * f
        
        ax.scatter(
            x,
            y,
            marker="o",
            c=xray_times[mask], 
            cmap=cmap, 
            vmin=bins[0],
            vmax=bins[-1]
        )
        max_x = f
        
    if np.logical_and(lower > tnt, lower < 160.):
        print(lower)
        x_range = np.array([6.687001 * 10 ** 9, 10**32.])
#         ax.plot(x_range, radio_photon(x_range/(10.**9))/conversion, color="k", linestyle=":")
        ax.plot(x_range, fit_g(x_range/(10.**9))/conversion, color="k", linestyle=":")
        ax.fill_between(x_range, fit_g(x_range/(10.**9), 2)/conversion, fit_g(x_range/(10.**9), -2)/conversion, color="k", linestyle=":", alpha=0.1)
        ax.fill_between(x_range, fit_g(x_range/(10.**9), 1)/conversion, fit_g(x_range/(10.**9), -1)/conversion, color="k", linestyle=":", alpha=0.1)

        #         ax.plot(x_range, fit_g(x_range/(10.**9), -2)/conversion, color="k", linestyle=":")

#         ax_inset = zoomed_inset_axes(ax, 10, loc="center")
    
    
#         ax_inset = plt.axes(
#             [0.2, 0.3, 0.3, 0.4]
#         )
    
#         ax_inset.scatter(1.4 * 10**9*np.ones_like(meerkat_data["flux_mJy"]), convert_radio(meerkat_data["flux_mJy"], 1.4)/conversion, 
#             marker="o", 
#             c=meerkat_times,
#             cmap=cmap, 
#             vmin=bins[0],
#             vmax=bins[-1]
#         )
        
#         ax.scatter(
#             vla_data["frequency"] * 10 ** 9, 
#             convert_radio(vla_data["flux"], vla_data["frequency"])/conversion, 
#             marker="o", 
#             c=radio_times, 
#             cmap=cmap, 
#             vmin=bins[0],
#             vmax=bins[-1]
#         )

#         ax_inset.set_ticklabel_visible(False)
#         ax_inset.set_ticks_visible(False)
#         ax.mark_inset_axes(ax_inset)
#         ax.connect_inset_axes(ax_inset, 'upper left')
#         ax.connect_inset_axes(ax_inset, 'upper right')
#         ax.connect_inset_axes(ax_inset, 'lower left')
#         ax_inset.set(
#             xlim=(1*10**9, 15 * 10 ** 9),
#             ylim=(10.**38, 10.**39.3), 
#             xscale="log", 
#             yscale="log"
#         )

   
    
#     if len(fs) > 2:
        
#         grad = (np.log(max(lums)) - np.log(min(lums)))/(np.log(max(fs)) - np.log(min(fs)))
                
#         def f(x):
#             return np.exp(np.log(min(lums)) + grad * (np.log(x) - np.log(min(fs))))

        
#         print(lower, grad, f(1.4*10**9), min_x)
        
#         x = [min_x, max_x]

#         plt.plot(
#             x,
#             f(x),
#             linestyle=":", 
#             color=matplotlib.cm.get_cmap(cmap)((np.mean([lower, upper]) - bins[0])/time_scale)
#         )

#         index_times.append(np.mean([lower, upper]))
#         photon_index.append(grad)
    return ax
    
# fig, ax = plt.subplots()
# xdata, ydata = [], []
# ln, = plt.plot([], [], 'ro')

# def init():
#     ax.set_xlim(0, 2*np.pi)
#     ax.set_ylim(-1, 1)
#     return ln,

# def update(frame):
#     xdata.append(frame)
#     ydata.append(np.sin(frame))
#     ln.set_data(xdata, ydata)
#     return ln,

ani = FuncAnimation(fig, update_plot, frames=frames,
                    blit=False)
ani.save("BranThroughTime.mp4")

# for j in frames:
#     update_plot(j)
#     camera.snap()

# anim = camera.animate(blit=False)
# anim.save("tester.html")

In [ ]:
plt.figure(figsize=(12, 10))

ax1 = plt.subplot2grid((4, 1), (0, 0), colspan=3, rowspan=3)
ax1b = ax1.twinx()

# Plot luminosity

for band in sorted(bands):
    if band in bands:
        c = colors[band]
        data = df[df["band"] == band]
        wl = bands[band].to("m")
        f = (const.c / wl).to("GHz")
        flux = conversion * data["lum"]
        f_convert = f.to("Hz").value
        lum = data["lum"]
        ax1b.errorbar(data["#day_since_peak"], lum, yerr=data["err_lum"]/f_convert, color=c,  fmt='o', label=band)
        ax1.errorbar(data["#day_since_peak"], flux, yerr=conversion *data["err_lum"]/f_convert, color=c,  fmt='o', label=band)

ax1.plot(meerkat_data["#mjd"] - t_peak_mjd.mjd, convert_radio(meerkat_data["flux_mJy"], 1.4), marker="o", color="violet", label="1.4 GHz")
ax1b.plot(meerkat_data["#mjd"] - t_peak_mjd.mjd, convert_radio(meerkat_data["flux_mJy"], 1.4)/conversion, marker="o", color="violet", label="1.4 GHz")

for frequency in [5.6, 10.2]:
    data = vla_data[abs(vla_data["frequency"] - frequency) < 0.5]
    data = data.sort_values("mjd")
    ax1.errorbar(data["mjd"]- t_peak_mjd.mjd, convert_radio(data["flux"], frequency),  yerr=convert_radio(data["flux_err"], frequency)/(frequency * 10**9), marker="o", label="{0} Ghz".format(frequency))
    ax1b.errorbar(data["mjd"]- t_peak_mjd.mjd, convert_radio(data["flux"], frequency)/conversion,  yerr=convert_radio(data["flux_err"], frequency)/(frequency * 10**9)/conversion, marker="o", label="{0} Ghz".format(frequency))

ax1.errorbar(xray_data["#MJD"] - t_peak_mjd.mjd, xray_data["flux"], yerr=xray_data["flux_err"], color="k", marker = "o")
# ax1.errorbar(xray_data["#MJD"] - t_peak_mjd.mjd, 10**xray_data["log_lum"], yerr=10**xray_data["log_lum_err"], label="X-Ray (0.3-10 keV)", color="k", marker="o")

ax1.set_ylabel(r"$\nu F_{\nu}$ [erg cm$^{-2}$ s$^{-1}$]", fontsize=12)
ax1b.set_ylabel(r"$\nu L_{\nu}$ [erg s$^{-1}$]", fontsize=12)
ax1.legend(fontsize=12)
ax1.set_yscale("log")
ax1b.set_yscale("log")
ax1.axvline(t_neutrino.mjd - t_peak_mjd.mjd, color="k", linestyle=":", label="IC191001A")
ax1.tick_params(axis='both', which='major', labelsize=12)
ax1b.tick_params(axis='both', which='major', labelsize=12)

ax2 = plt.subplot2grid((4, 1), (3, 0), colspan=3, rowspan=1, sharex=ax1)
ax2.scatter(index_times, photon_index, label="UV-optical")
ax2.set_ylabel(r"$\frac{d(Log(\nu L_{\nu}))}{d(Log(f))}$", fontsize=12)
plt.axhline(2.0, color="k", linestyle=":")
ax2.tick_params(axis='both', which='major', labelsize=12)
plt.legend(fontsize=12)

plt.tight_layout()
plt.subplots_adjust(hspace=.0)


In [ ]:
radio_times = vla_data.sort_values("mjd")["mjd"] - t_peak_mjd.mjd

plt.figure(figsize=(12,10))
ax = plt.subplot(111)

t0 = -15

for time in list(set(radio_times)):
    mask = radio_times == time
    ax.errorbar(
        vla_data["frequency"][mask] * 10 ** 9, 
        vla_data["flux"][mask]/(time - t0), 
        marker="o", 
        label = f"{time:.0f} days post-peak",
        yerr=vla_data["flux_err"][mask]/(time - t0),
        fmt = "",
        linestyle=" "
    )
ax.tick_params(axis='both', which='major', labelsize=12)
plt.xscale("log")
plt.yscale("log")
plt.legend(fontsize=12)
plt.ylabel("Flux (mJy) / (Time since peak)", fontsize=12)
plt.xlabel("f (Hz)", fontsize=12)
plt.title(f"Flux scaled with time since {t0} days")